In [279]:
import pandas as pd
import numpy as np
import random
from random import choices
from random import choice
import warnings
import math
from joblib import Parallel, delayed
import time
from faker import Faker
# from random import randint, randrange
import datetime

In [273]:
crosswalk = pd.read_csv('GeoCrosswalk.csv')
crosswalk_Origin = pd.read_csv('GeoCrosswalk_origin.csv')
crosswalk_Destination = pd.read_csv('GeoCrosswalk_destination.csv')
df_city = pd.read_csv('used_city.csv')
df_flight = pd.read_csv('EuropeFlightTraffic/EUFlight.csv')
df_route = pd.read_csv('df_route_routingV2.csv')
df_flight_no_POI = pd.read_csv('flights_no_POI.csv')

In [274]:
def cleanFlight(df_flight):
    df_flight['bookings'] = df_flight['bookings'].str.replace("[", "")
    df_flight['bookings'] = df_flight['bookings'].str.replace("]", "")
    df_flight['bookings'] = df_flight['bookings'].str.replace("'", "")
    df_flight['bookings'] = df_flight['bookings'].str.replace(" ", "")
    df_flight['bookings'] = df_flight['bookings'].str.split(",")
    df_flight['firstseen'] = pd.to_datetime(df_flight['firstseen'],utc=True)
    df_flight['lastseen'] = pd.to_datetime(df_flight['lastseen'],utc=True)
    df_flight['day'] = pd.to_datetime(df_flight['day'],utc=True)
    return df_flight

In [275]:
df_flight_no_POI = cleanFlight(df_flight_no_POI)

In [267]:
df_flight_no_POI

,Unnamed: 0,flight_id,callsign,number,icao24,registration,typecode,origin,destination,firstseen,...,Country_A,capacity,routeOperator,origin_outlier,destination_outlier,LegAirport_ICAO_outlier,occupancy,bookings,outPNRs,retPNRs
0,0,0,CSN461,NaN,780aec,B-2042,B77L,KORD,EDDF,2019-02-28 01:25:50+00:00,...,China,320.0,KORD-EDDF-CSN,Normal,Normal,Normal,0,[],2,0
1,1,2,CCA855,CA855,7804ca,B-6505,A332,OMDB,EGLL,2019-02-28 04:22:58+00:00,...,China,293.0,OMDB-EGLL-CCA,Normal,Normal,Normal,0,[],2,0
2,2,3,AAR501,NaN,71c254,HL8254,B772,KORD,LFPG,2019-02-28 06:14:42+00:00,...,South Korea,312.0,KORD-LFPG-AAR,Normal,Normal,Normal,0,[],2,0
3,3,10,THA926,NaN,885179,HS-TKY,B77W,EKCH,EDDF,2019-02-28 13:17:58+00:00,...,Thailand,212.0,EKCH-EDDF-THA,Normal,Normal,Normal,0,[],2,0
4,4,20,AFR293,AF293,394a09,F-GSQJ,B77W,RJTT,LFPG,2019-02-28 15:06:52+00:00,...,France,212.0,RJTT-LFPG-AFR,Normal,Normal,Normal,0,[],2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137276,137276,223526,DLH2EN,NaN,3c6610,D-AIPP,A320,EDDF,EGBB,2019-03-31 21:28:42+00:00,...,Germany,150.0,EDDF-EGBB-DLH,Normal,Normal,Normal,27,"[WAW-BHX-leisure-ID39, WAW-BHX-leisure-ID47, W...",287,12
137277,137277,223529,EZY72RF,NaN,400a69,G-EZEB,A319,LFBO,EGKK,2019-03-31 21:29:48+00:00,...,United Kingdom,124.0,LFBO-EGKK-EZY,Normal,Normal,Normal,22,"[TLS-LGW-group-ID4, TLS-ARN-business-ID5, TLS-...",213,9
137278,137278,223532,NAX2PC,NaN,47947c,LN-NIH,B738,ENGM,ESSA,2019-03-31 21:32:13+00:00,...,Norway,162.0,ENGM-ESSA-NAX,Normal,Normal,Normal,5,"[OSL-LHR-business-ID3, OSL-LHR-leisure-ID196]",49,2
137279,137279,223538,BAW89ZV,NaN,406b5b,G-GATL,A320,LIPZ,EGKK,2019-03-31 22:00:09+00:00,...,United Kingdom,150.0,LIPZ-EGKK-BAW,Normal,Normal,Normal,8,"[VCE-LGW-business-ID13, VCE-DUB-group-ID4, VCE...",70,3


# Create POI HH

In [43]:
num_HH = 10
def createHH(num_HH, cw, df_city):
    cw = cw[cw['region'] == 'Europe']
    list_of_POIHH = []
    for i in range(num_HH):
        HHISO_ = choice(df_city['HH_ISO'].tolist())
        HHID_ = "POI"+"_"+str(i)
        GenderHOH_ = choice(["M", "F"])
        HHType_ = choice(["T1", "T2"])
        if HHType_ == 'T1':
            AgeHOH_ = choice(list(range(5, 16+1)))
        elif HHType_ == 'T2':
            AgeHOH_ = choice(list(range(28, 55)))
        if HHType_ == "T1":
            SizeHH_ = 1
        elif HHType_ == "T2":
            SizeHH_ = 2
        Lang_ = choice(cw[cw['HH_ISO'] == HHISO_]['Lang'].unique().tolist())
        locale_gen = Faker(Lang_)
        Surname_ = (locale_gen.last_name())
        Address_ = (locale_gen.address())
        PostCode_ = (locale_gen.postcode())
        Country_ = (locale_gen.current_country())
        row = [i, HHISO_, HHID_, GenderHOH_, AgeHOH_, SizeHH_, HHType_, Lang_, Surname_, Address_, PostCode_, Country_]
        list_of_POIHH.append(row)
    df_HH = pd.DataFrame(list_of_POIHH, columns=['HH_num', 'HH_ISO', 'HHID', 'GenderHOH', 'AgeHOH', 'SizeHH', 'HHType', 'Lang', 'Surname', 'Address', 'PostCode', 'Country'])
    return df_HH

In [171]:
def destinationAssign(df_HH, df_flight):
    IATA_D = []
    not_include = ['GRC', 'SVN']
    df_city_lookup = pd.DataFrame(columns=['HH_ISO', 'IATA_D', 'Cap_D'])
    df_handler = pd.DataFrame(columns=['HH_ISO', 'IATA_D', 'Cap_D'])
    df_HH = df_HH.replace(['BLR', 'BGR', 'GEO', 'LTU', 'SRB', 'UKR', 'MLT', 'MNE'],
                          ['RUS', 'TUR', 'TUR', 'POL', 'HUN', 'RUS', 'ITA', 'GRC'])
    df_HH.drop(df_HH[df_HH['HH_ISO'].isin(not_include)].index, inplace = True)
    for HH_ISO in df_HH['HH_ISO'].unique():
        des = df_flight[(df_flight['HH_ISO_O']!=HH_ISO) & (df_flight['HH_ISO_D']!=HH_ISO)].groupby('IATA_D')['capacity'].sum().reset_index()
        destination = list(des['IATA_D'])
        cap_des = list(des['capacity']/des['capacity'].sum())
        if HH_ISO == 'GRC':
            origin = ['ATH']
            cap_org = [1.0]        
        df_handler['HH_ISO'] = pd.Series(HH_ISO)
        df_handler['IATA_D'] = pd.Series([destination])
        df_handler['Cap_D'] = pd.Series([cap_des])     
        df_city_lookup = df_city_lookup.append(df_handler)
    for i in df_HH['HH_ISO']:
        x = choices(df_city_lookup[df_city_lookup['HH_ISO'] == i]['IATA_D'][0], 
                    df_city_lookup[df_city_lookup['HH_ISO'] == i]['Cap_D'][0])
        IATA_D.append(x[0])
    return IATA_D

In [163]:
def originalCityAssign(df_HH, df_flight):
    IATA_O = []
    IATA_D = []
    not_include = ['GRC', 'SVN']
#new behavioral function
#     df_city_lookup = pd.DataFrame(columns=['HH_ISO', 'IATA_O', 'Cap_O', 'IATA_D', 'Cap_D'])
#     df_handler = pd.DataFrame(columns=['HH_ISO', 'IATA_O', 'Cap_O', 'IATA_D', 'Cap_D'])
    df_city_lookup = pd.DataFrame(columns=['HH_ISO', 'IATA_O', 'Cap_O'])
    df_handler = pd.DataFrame(columns=['HH_ISO', 'IATA_O', 'Cap_O'])
    df_HH = df_HH.replace(['BLR', 'BGR', 'GEO', 'LTU', 'SRB', 'UKR', 'MLT', 'MNE'],
                          ['RUS', 'TUR', 'TUR', 'POL', 'HUN', 'RUS', 'ITA', 'GRC'])
    
    df_HH.drop(df_HH[df_HH['HH_ISO'].isin(not_include)].index, inplace = True)
    for HH_ISO in df_HH['HH_ISO'].unique():
        org = df_flight.groupby(['HH_ISO_O','IATA_O'])['capacity'].sum().reset_index()
        org = org[org['HH_ISO_O'] == HH_ISO]
        origin = list(org['IATA_O'])
        cap_org = list(org['capacity']/org['capacity'].sum())
#new behavioral function                          
#         des = df_flight[(df_flight['HH_ISO_O']!=HH_ISO) & (df_flight['HH_ISO_D']!=HH_ISO)].groupby('IATA_D')['capacity'].sum().reset_index()
#         destination = list(des['IATA_D'])
#         cap_des = list(des['capacity']/des['capacity'].sum())
        if HH_ISO == 'GRC':
            origin = ['ATH']
            cap_org = [1.0]        
        df_handler['HH_ISO'] = pd.Series(HH_ISO)
        df_handler['IATA_O'] = pd.Series([origin])
        df_handler['Cap_O'] = pd.Series([cap_org])
#new behavioral function
#         df_handler['IATA_D'] = pd.Series([destination])
#         df_handler['Cap_D'] = pd.Series([cap_des])        
        df_city_lookup = df_city_lookup.append(df_handler)
    
    for i in df_HH['HH_ISO']:
        x = choices(df_city_lookup[df_city_lookup['HH_ISO'] == i]['IATA_O'][0], 
                    df_city_lookup[df_city_lookup['HH_ISO'] == i]['Cap_O'][0])
#new behavioral function
#         y = choices(df_city_lookup[df_city_lookup['HH_ISO'] == i]['IATA_D'][0], 
#                     df_city_lookup[df_city_lookup['HH_ISO'] == i]['Cap_D'][0])
        IATA_O.append(x[0])
#         IATA_D.append(y[0])
    
    df_HH.insert(2, 'IATA_O', IATA_O)
#     df_HH.insert(3, 'IATA_D', IATA_D)
    
    return df_city_lookup, df_HH
    

In [204]:
def behaviour(df_HH, df_flight, behaviour_num):
    df_behaviour = pd.DataFrame()
    print('df_HH.shape: ',df_HH.shape)
    column_name = []
    df_HH.columns
    for i in range(behaviour_num):
        destination = destinationAssign(df_HH, df_flight)
        traveller_type = ['POI'] * df_HH.shape[0]
        travel_num = df_HH['SizeHH']
        behaviour_list = [list(x) for x in zip(destination, traveller_type, travel_num)]
        df_behaviour['Behaviour_'+str(i+1)] = pd.Series(behaviour_list)
        print('df_behaviour_shape: ', df_behaviour.shape)
        
    df_behaviour.insert(0, 'HHID', df_HH['HHID'].tolist())
    df_behaviour_col = list(df_behaviour.columns)
    del df_behaviour_col[0]
    column_name = list(list(df_HH.columns)+df_behaviour_col)
    df_behaviour_complete = df_HH.merge(df_behaviour, on = 'HHID', how = 'left')
#     df_behaviour_complete = travelType(df_behaviour_complete, behaviour_num)
    
    return df_behaviour, df_behaviour_complete

In [187]:
def travelType(df, behaviour_num):
    for i in range(behaviour_num):
        colName      = 'Behaviour_'+str(i+1)
        func         = np.vectorize(conditions)
        trType = func([el[1] for el in df[colName]], list(df['SizeHH']), list(df['HHType']))
        travelType = list(trType[0])
        NP = list(trType[1])

    # Update the inner lists in the 'list_column' with different values
        df[colName] = [sublist + [value] for sublist, value in zip(df[colName], travelType)]
        df[colName] = [sublist + [value] for sublist, value in zip(df[colName], NP)]
        
    
    return df

In [189]:
def conditions(purpose, Nparty, HHType):
    if   purpose == 'business':
        return 'business', 0
    elif purpose == 'leisure':
        if Nparty == 1: 
            return 'solo', Nparty
        else:
            if HHType == '2B':
                return 'couple', Nparty
            if HHType == '2A':
                tipe = choices(['family', 'couple'], [0.5, 0.5])
                if tipe[0] == 'family':
                    return 'family', Nparty
                else:
                    return 'couple', 2
            elif HHType == '1A':
                return 'family', Nparty
            else:
                return 'family', Nparty
    elif purpose == 'group':
#         z = choices([3,4,5,6,7], [0.518733, 0.209850, 0.211782, 0.029883, 0.029752])
        return 'group', 0  

In [197]:
def extract_group_value(row):
    return row[0]+'-'+row[1] if len(row) > 0 else None

def extract_OD(row):
    return row[0:7] if len(row) > 0 else None

In [224]:
def bookInit(df_behaviour_complete, df_route):
    df_behaviour_complete['group_value'] = df_behaviour_complete['IATA_O']+"-"+df_behaviour_complete['Behaviour_1'].apply(extract_group_value)
    s = df_behaviour_complete.groupby('group_value').cumcount().add(0).astype(str)
    df_behaviour_complete['group_value'] += ('-ID' + s)
    
    list_of_passengers = []
    for i in range(len(df_behaviour_complete['HHID'].unique())):
        LOP = []
        for j in range(df_behaviour_complete['Behaviour_1'][i][2]):
            passengers = df_behaviour_complete['HHID'][i]+"_"+str(j+1)
            LOP.append(passengers)
        list_of_passengers.append(LOP)
        
    df_POI = pd.DataFrame(list(zip(list(df_behaviour_complete['group_value']),list_of_passengers)), columns = ['initID', 'list_of_passengers'])
    df_POI['route'] = df_POI['initID'].apply(extract_OD)
    df_POI['num_in_party'] = df_POI['list_of_passengers'].str.len()
    
    df_route = df_route[['route', 'fixRoute', 'Leg_fixRoute', 'LegHH_ISO']].copy()
    df_POI = df_POI.merge(df_route, on = 'route', how = 'left')
    
    long_stay = [np.nan]*df_POI.shape[0]
    df_POI.insert(2, "stayDay", long_stay)
    handler = df_POI['fixRoute'].str.split("->", expand = True).values.tolist()
    handler = [cleaned for sublist in handler
               if (cleaned := [elem for elem in sublist if elem is not None])]
    df_POI['Airport_Itinerary_Out'] = handler
    df_POI['full_routes'] = df_POI['Airport_Itinerary_Out'].apply(complete_routing)
    return df_POI

def complete_routing(sublist):
    if len(sublist) >= 2:
        pairs = [f"{sublist[i]}-{sublist[i+1]}" for i in range(len(sublist)-1)]
        return pairs
    return []
    return df_POI

def outboundDay(df, df_flight):
    date = []
    flight_sched = df_flight.groupby(['day'])['capacity'].sum().reset_index()
    flight_sched['prob'] = flight_sched['capacity']/(flight_sched['capacity'].sum())
    for i in range(len(df)):
        x = choices(flight_sched['day'].tolist(), flight_sched['prob'].tolist())[0]
        date.append(x)
    df['outboundDate'] = date
    df['returnDate'] = pd.to_datetime(df['outboundDate'])+pd.to_timedelta(df['stayDay'], unit='D')
    return df

In [256]:
def outboundItinerary(df, df_flight):
    x = []
    df = df.dropna(subset=['Leg_fixRoute'])
    for index, row in df.iterrows():
        ID = row['initID']
        print(ID)
        LOP = row['list_of_passengers']
        numLeg = int(row['Leg_fixRoute'])
        initDT = row['outboundDate']
#         print('ID: ', ID, ' numLeg: ', numLeg, ' outboundDate: ', initDT)
        numPrty = row['num_in_party']
        y = []
        for i in range(numLeg):
            LEG = row['full_routes'][i]
            if i < 1:
                chosenFlight, df_flight, nextFlight = searchFlight(ID, df_flight, initDT, LEG, numPrty)
            else:
                chosenFlight, df_flight, nextFlight = searchFlight(ID, df_flight, initDT, LEG, numPrty, nextFlight)
            
#             print('chosenFlight: ', chosenFlight, ' flightSched: ', df_flight[df_flight['flight_id'] == chosenFlight]['day'], ' nextFlight: ', nextFlight, )    
            y.append(chosenFlight)
        x.append(y)
    df_flight['outPNRs'] = df_flight['bookings'].apply(len)
    df['outboundFlights'] = x
    return df, df_flight

In [297]:
def returnItinerarty(df, df_flight):
    df['Airport_Itinerary_Ret'] = df['Airport_Itinerary_Out'].apply(lambda x: x[::-1])
    df['return_routes'] = df['Airport_Itinerary_Ret'].apply(complete_routing)
    x = []
#     for index, row in df.iterrows():
#         ID = row['initID']
#         LOP = row['list_of_passengers']
#         numLeg = int(row['Leg_fixRoute'])
#         initDT = row['returnDate']
#         numPrty = row['num_in_party']
#         y = []
#         for i in range(numLeg):
#             LEG = row['return_routes'][i]
#             if i < 1:
#                 chosenFlight, df_flight, nextFlight = searchFlight(ID, df_flight, initDT, LEG, numPrty)
#             else:
#                 chosenFlight, df_flight, nextFlight = searchFlight(ID, df_flight, initDT, LEG, numPrty, nextFlight)
                
#             y.append(chosenFlight)
#         x.append(y)
    a = Parallel(n_jobs=2)(delayed(retret)(row, df_flight) for index, row in df.iterrows())
    print(a)
    x.append(a)
    df_flight['retPNRs'] = df_flight['bookings'].apply(len)
    df['returnFlights'] = a
    return df, df_flight

In [295]:
def retret(row, df_flight):
    ID = row['initID']
    LOP = row['list_of_passengers']
    numLeg = int(row['Leg_fixRoute'])
    initDT = row['returnDate']
    numPrty = row['num_in_party']
    y = []
    for i in range(numLeg):
        LEG = row['return_routes'][i]
        if i < 1:
            chosenFlight, df_flight, nextFlight = searchFlight(ID, df_flight, initDT, LEG, numPrty)
        else:
            chosenFlight, df_flight, nextFlight = searchFlight(ID, df_flight, initDT, LEG, numPrty, nextFlight)
                
        y.append(chosenFlight)
    print(y)
    return y

In [300]:
df_HH

,HH_num,HH_ISO,IATA_O,HHID,GenderHOH,AgeHOH,SizeHH,HHType,Lang,Surname,Address,PostCode,Country
0,0,IRL,DUB,POI_0,M,36,2,T2,ga_IE,Ó Lainn,"11971 Mac Thámhais Passage\nIngridton, AK 53359",17045,Republic of Ireland
1,1,DEU,MUC,POI_1,M,13,1,T1,de_DE,Zirme,Irmtraud-Dietz-Allee 0\n50901 Wetzlar,30274,Germany
2,2,ROU,OTP,POI_2,F,29,2,T2,ro_RO,Tudor,"Intrarea Ghenadie Manole\nAnina, 245855",904976,Romania
3,3,SWE,ARN,POI_3,M,46,2,T2,sv_SE,Eriksson,Idrottsvägen 422\n28628 Sundsvall,54847,Sweden
4,4,LVA,RIX,POI_4,F,9,1,T1,NaN,Arroyo,"0769 Brown Mall\nEricmouth, SC 51648",24248,United States
5,5,CHE,ZRH,POI_5,M,51,2,T2,fr_CH,Pasquier,rue Nusslé\n4674 Chatriand-sur-Pachoud,5788,Switzerland
6,6,BEL,BRU,POI_6,F,36,2,T2,nl_BE,Burke,Tammybaan 7\n3658\nHuldenberg,6450,Belgium
7,7,SVK,BTS,POI_7,M,47,2,T2,sk_SK,Harmon,Žitná 90\n010 21 Orovnica,946 93,Slovakia
8,8,HRV,ZAG,POI_8,M,32,2,T2,hr_HR,Modrić,Ilica 1b/8\n80925 Novska,89437,Croatia
9,9,ROU,OTP,POI_9,M,9,1,T1,ro_RO,Mazilescu,"Aleea Olimpia Toma\nSovata, 255504",586889,Romania


In [252]:
def searchFlight(ID, df_flight, date, LEG, numPrty, time = None):
    if time is None:
        df_f_filtered = df_flight[df_flight['day'] == date]
    else:
        time_limit = time+pd.to_timedelta(8, unit='h')
#         print('time limit: ', time_limit)
        df_f_filtered = df_flight[df_flight['firstseen'] <= time_limit]
        df_f_filtered = df_f_filtered[df_f_filtered['firstseen'] >= time]
    
    df_f_filtered = df_f_filtered[df_f_filtered['LegAirport_IATA'] == LEG]
    df_f_filtered = df_f_filtered[(df_f_filtered['occupancy']+numPrty <= df_f_filtered['capacity'])]
    
    flights = df_f_filtered['flight_id'].tolist()
    weight = df_f_filtered['capacity']/(df_f_filtered['capacity'].sum()).tolist()
    if len(flights) <1:
        chosenFlight = float("NAN")
    else:
        chosenFlight = choices(flights, weight)[0]
        print(chosenFlight)
        df_flight.loc[df_flight['flight_id'] == chosenFlight, 'occupancy'] += numPrty
        df_flight.loc[df_flight['flight_id'] == chosenFlight, 'bookings'].iloc[0].append(ID)
        
    if math.isnan(chosenFlight):
        nextFlight = pd.Timestamp(pd.NaT, tz = 'UTC')
    else :
        nextFlight = pd.Timestamp(df_flight[df_flight['flight_id'] == chosenFlight]['lastseen'].values[0], tz = 'UTC')

    return chosenFlight, df_flight, nextFlight

In [233]:
df_city_lookup, df_HH = originalCityAssign(z, df_flight)
df_behaviour, df_behaviour_complete = behaviour(df_HH, df_flight, 1)
POI = bookInit(df_behaviour_complete, df_route)
POI = outboundDay(POI, df_flight)
POI

df_HH.shape:  (10, 13)
df_behaviour_shape:  (10, 1)


/Users/muhammadfathifadlian/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/timedeltas.py:939: RuntimeWarning: invalid value encountered in cast
  base = data.astype(np.int64)
/Users/muhammadfathifadlian/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/timedeltas.py:943: RuntimeWarning: invalid value encountered in cast
  data = (base * m + (frac * m).astype(np.int64)).view("timedelta64[ns]")


,initID,list_of_passengers,stayDay,route,num_in_party,fixRoute,Leg_fixRoute,LegHH_ISO,Airport_Itinerary_Out,full_routes,outboundDate,returnDate
0,DUB-JFK-POI-ID0,"[POI_0_1, POI_0_2]",NaN,DUB-JFK,2,NaN,NaN,NaN,"[nan, nan, nan]","[nan-nan, nan-nan]",2019-03-25 00:00:00+00:00,NaT
1,MUC-LGW-POI-ID0,[POI_1_1],NaN,MUC-LGW,1,MUC->LGW,1.0,DEU-GBR,"[MUC, LGW]",[MUC-LGW],2019-03-20 00:00:00+00:00,NaT
2,OTP-LGW-POI-ID0,"[POI_2_1, POI_2_2]",NaN,OTP-LGW,2,OTP->LGW,1.0,ROU-GBR,"[OTP, LGW]",[OTP-LGW],2019-03-10 00:00:00+00:00,NaT
3,ARN-BRS-POI-ID0,"[POI_3_1, POI_3_2]",NaN,ARN-BRS,2,ARN->AMS->BRS,2.0,SWE-GBR,"[ARN, AMS, BRS]","[ARN-AMS, AMS-BRS]",2019-03-21 00:00:00+00:00,NaT
4,RIX-NOC-POI-ID0,[POI_4_1],NaN,RIX-NOC,1,RIX->STN->NOC,2.0,LVA-IRL,"[RIX, STN, NOC]","[RIX-STN, STN-NOC]",2019-03-07 00:00:00+00:00,NaT
5,ZRH-DUS-POI-ID0,"[POI_5_1, POI_5_2]",NaN,ZRH-DUS,2,NaN,NaN,NaN,"[nan, nan, nan]","[nan-nan, nan-nan]",2019-03-12 00:00:00+00:00,NaT
6,BRU-DEL-POI-ID0,"[POI_6_1, POI_6_2]",NaN,BRU-DEL,2,NaN,NaN,NaN,"[nan, nan, nan]","[nan-nan, nan-nan]",2019-03-29 00:00:00+00:00,NaT
7,BTS-ZRH-POI-ID0,"[POI_7_1, POI_7_2]",NaN,BTS-ZRH,2,NaN,NaN,NaN,"[nan, nan, nan]","[nan-nan, nan-nan]",2019-03-04 00:00:00+00:00,NaT
8,ZAG-HAM-POI-ID0,"[POI_8_1, POI_8_2]",NaN,ZAG-HAM,2,ZAG->LHR->HAM,2.0,HRV-DEU,"[ZAG, LHR, HAM]","[ZAG-LHR, LHR-HAM]",2019-03-26 00:00:00+00:00,NaT
9,OTP-LHR-POI-ID0,[POI_9_1],NaN,OTP-LHR,1,OTP->LHR,1.0,ROU-GBR,"[OTP, LHR]",[OTP-LHR],2019-03-31 00:00:00+00:00,NaT


In [298]:
book_POI, flight_POI = outboundItinerary(POI, df_flight_no_POI)
book_POI_2, flight_POI_2 = returnItinerarty(book_POI, flight_POI)

MUC-LGW-POI-ID0
141248
OTP-LGW-POI-ID0
74802
ARN-BRS-POI-ID0
151717
153090
RIX-NOC-POI-ID0
53398
ZAG-HAM-POI-ID0
184710
185978
OTP-LHR-POI-ID0
217108


/var/folders/j0/bkdgfzfs5bx0c6wgphwnk6kr0000gq/T/ipykernel_65672/3082089013.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['outboundFlights'] = x
/var/folders/j0/bkdgfzfs5bx0c6wgphwnk6kr0000gq/T/ipykernel_65672/616169053.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Airport_Itinerary_Ret'] = df['Airport_Itinerary_Out'].apply(lambda x: x[::-1])
/var/folders/j0/bkdgfzfs5bx0c6wgphwnk6kr0000gq/T/ipykernel_65672/616169053.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
[[nan], [nan], [nan, nan], [nan, nan], [nan, nan], [nan]]


/var/folders/j0/bkdgfzfs5bx0c6wgphwnk6kr0000gq/T/ipykernel_65672/616169053.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['returnFlights'] = a


In [489]:
def populate_1(i, df_HH):
    P_ID = []
    P_AGE_ = []
    P_GENDER = []
    HHID = []
    AgeRange = []
    AgeGroup = []
    for j in range(df_HH.SizeHH[i]):
#         print(j)
        P_ID.append(df_HH.HHID[i]+"_"+str(j+1))
#         print("HHTYPE: ",P_ID, df_HH.HHType[i])
        HHID.append(df_HH.HHID[i])
        minAge = 5
        maxAge = 17
        if df_HH.HHType[i] == 'T1':
            P_AGE = df_HH.AgeHOH[i]
            P_AGE_.append(P_AGE)
            P_GENDER.append(df_HH.GenderHOH[i])
#             print(P_ID, P_AGE)        
        elif df_HH.HHType[i] == 'T2':
            if j == 0:
                P_AGE = df_HH.AgeHOH[i]
                P_AGE_.append(P_AGE)
                P_GENDER.append(df_HH.GenderHOH[i])
#                 print(P_ID, P_AGE)        
            if 0<j:
                P_AGE = random.randrange(minAge, maxAge)
                P_AGE_.append(P_AGE)
                P_GENDER.append(choices(['M', 'F'], [0.3, 0.7])[0])
#                 print(P_ID, P_AGE)        
        if 0<=P_AGE<=4:
            AgeRange.append('0-4')
            AgeGroup.append('AGE1')
        elif 5<=P_AGE<=9:
            AgeRange.append('5-9')
            AgeGroup.append('AGE2')
        elif 10<=P_AGE<=14:
            AgeRange.append('10-14')
            AgeGroup.append('AGE3')
        elif 15<=P_AGE<=19:
            AgeRange.append('15-19')
            AgeGroup.append('AGE4')
        elif 20<=P_AGE<=24:
            AgeRange.append('20-24')
            AgeGroup.append('AGE5')
        elif 25<=P_AGE<=29:
            AgeRange.append('25-29')
            AgeGroup.append('AGE6')
        elif 30<=P_AGE<=34:
            AgeRange.append('30-34')
            AgeGroup.append('AGE7')
        elif 35<=P_AGE<=39:
            AgeRange.append('35-39')
            AgeGroup.append('AGE8')
        elif 40<=P_AGE<=44:
            AgeRange.append('40-44')
            AgeGroup.append('AGE9')
        elif 45<=P_AGE<=49:
            AgeRange.append('45-49')
            AgeGroup.append('AGE10')
        elif 50<=P_AGE<=54:
            AgeRange.append('50-54')
            AgeGroup.append('AGE11')
        elif 55<=P_AGE<=59:
            AgeRange.append('55-59')
            AgeGroup.append('AGE12')
        elif 60<=P_AGE<=64:
            AgeRange.append('60-64')
            AgeGroup.append('AGE13')
        elif 65<=P_AGE<=69:
            AgeRange.append('65-69')
            AgeGroup.append('AGE14')
        elif 70<=P_AGE<=74:
            AgeRange.append('70-74')
            AgeGroup.append('AGE15')
        elif 75<=P_AGE<=79:
            AgeRange.append('75-79')
            AgeGroup.append('AGE16')
        elif 80<=P_AGE<=84:
            AgeRange.append('80-84')
            AgeGroup.append('AGE17')
        elif 85<=P_AGE<=89:
            AgeRange.append('85-89')
            AgeGroup.append('AGE18')
        elif 90<=P_AGE<=94:
            AgeRange.append('90-94')
            AgeGroup.append('AGE19')
        elif 95<=P_AGE<=99:
            AgeRange.append('95-99')
            AgeGroup.append('AGE20')
        elif 100<=P_AGE:
            AgeRange.append('100+')
            AgeGroup.append('AGE21')
        else:
            AgeRange.append('Unidentified')
            AgeGroup.append('Unidentified')
#     print(i)
#     print("idon", HHID, P_ID, P_AGE, P_GENDER)  
#     print("idon")
    return HHID, P_ID, P_AGE_, P_GENDER, AgeRange, AgeGroup

        


SyntaxError: invalid syntax (3173749297.py, line 3)

In [490]:
it = df_HH.shape[0]
a = Parallel(n_jobs=5)(delayed(populate_1)(i, df_HH) for i in range(it))
HHID_list, P_ID_list, P_AGE_list, P_GENDER_list, AgeRange_list, AgeGroup_list = zip(*a)
P_ID_list = [item for sublist in P_ID_list for item in sublist]
P_AGE_list = [item for sublist in P_AGE_list for item in sublist]
# P_AGE_list = list(P_AGE_list)
P_GENDER_list = [item for sublist in P_GENDER_list for item in sublist]
HHID_list = [item for sublist in HHID_list for item in sublist]
AgeRange_list = [item for sublist in AgeRange_list for item in sublist]
AgeGroup_list = [item for sublist in AgeGroup_list for item in sublist]
dict = {'HHID':HHID_list, 'P_ID' : P_ID_list, 'P_AGE': P_AGE_list,'AgeRange': AgeRange_list,
        'AgeGroup':AgeGroup_list, 'P_GENDER': P_GENDER_list}
Person_data = pd.DataFrame(dict)
Combined = Person_data.merge(df_HH, on = 'HHID', how = 'left')
Combined

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


,HHID,P_ID,P_AGE,AgeRange,AgeGroup,P_GENDER,HH_num,HH_ISO,IATA_O,GenderHOH,AgeHOH,SizeHH,HHType,Lang,Surname,Address,PostCode,Country
0,POI_0,POI_0_1,36,35-39,AGE8,M,0,IRL,DUB,M,36,2,T2,ga_IE,Ó Lainn,"11971 Mac Thámhais Passage\nIngridton, AK 53359",17045,Republic of Ireland
1,POI_0,POI_0_2,15,15-19,AGE4,M,0,IRL,DUB,M,36,2,T2,ga_IE,Ó Lainn,"11971 Mac Thámhais Passage\nIngridton, AK 53359",17045,Republic of Ireland
2,POI_1,POI_1_1,13,10-14,AGE3,M,1,DEU,MUC,M,13,1,T1,de_DE,Zirme,Irmtraud-Dietz-Allee 0\n50901 Wetzlar,30274,Germany
3,POI_2,POI_2_1,29,25-29,AGE6,F,2,ROU,OTP,F,29,2,T2,ro_RO,Tudor,"Intrarea Ghenadie Manole\nAnina, 245855",904976,Romania
4,POI_2,POI_2_2,12,10-14,AGE3,M,2,ROU,OTP,F,29,2,T2,ro_RO,Tudor,"Intrarea Ghenadie Manole\nAnina, 245855",904976,Romania
5,POI_3,POI_3_1,46,45-49,AGE10,M,3,SWE,ARN,M,46,2,T2,sv_SE,Eriksson,Idrottsvägen 422\n28628 Sundsvall,54847,Sweden
6,POI_3,POI_3_2,7,5-9,AGE2,M,3,SWE,ARN,M,46,2,T2,sv_SE,Eriksson,Idrottsvägen 422\n28628 Sundsvall,54847,Sweden
7,POI_4,POI_4_1,9,5-9,AGE2,F,4,LVA,RIX,F,9,1,T1,NaN,Arroyo,"0769 Brown Mall\nEricmouth, SC 51648",24248,United States
8,POI_5,POI_5_1,51,50-54,AGE11,M,5,CHE,ZRH,M,51,2,T2,fr_CH,Pasquier,rue Nusslé\n4674 Chatriand-sur-Pachoud,5788,Switzerland
9,POI_5,POI_5_2,10,10-14,AGE3,M,5,CHE,ZRH,M,51,2,T2,fr_CH,Pasquier,rue Nusslé\n4674 Chatriand-sur-Pachoud,5788,Switzerland


In [493]:
FirstName = []
DOB = []
FreeEmail = []
WorkEmail = []
DOCS_ExpiryDate = []
PaymentInfo_VendorCode = []
PaymentInfo_ExpiryDate = []
PaymentInfo_AccountNbr = []
locale_gen = Faker(lang)

for i in range(Combined.shape[0]):
    lang = Combined.Lang[i]
    
#     FirstName
    if Combined.P_GENDER[i] == 'M':
        FirstName.append(locale_gen.first_name_male())
    else:
        FirstName.append(locale_gen.first_name_female())
    
#     Date OF Birth
    YOB = 2021-Combined.P_AGE[i]
    DOB.append(locale_gen.date_between_dates(date_start = datetime.date(YOB, 1,1),
                                            date_end = datetime.date(YOB, 12, 31)))
    
#     FreeEmail
    if Combined.P_AGE[i]<10:
        FreeEmail.append("-")
    else:
        FreeEmail.append(locale_gen.free_email())
        
    if Combined.P_AGE[i]<18:
        PaymentInfo_VendorCode.append("-")
        PaymentInfo_ExpiryDate.append("-")
        PaymentInfo_AccountNbr.append("-")
        WorkEmail.append("-")
    elif 18<=Combined.P_AGE[i]<=20:
        num = choices(['Y', 'N'], [0.75, 0.25])[0]
        if num == 'N':
            PaymentInfo_VendorCode.append("-")
            PaymentInfo_ExpiryDate.append("-")
            PaymentInfo_AccountNbr.append("-")
            WorkEmail.append("-")
        elif num == 'Y':
            PaymentInfo_VendorCode.append(locale_gen.credit_card_provider())
            PaymentInfo_ExpiryDate.append(locale_gen.date_between_dates(
                date_start = datetime.date(2022, 1, 1),
                date_end = datetime.date(2027, 1, 1)))
            PaymentInfo_AccountNbr.append(locale_gen.credit_card_number())
            WorkEmail.append("-")
    else:
        PaymentInfo_VendorCode.append(locale_gen.credit_card_provider())
        PaymentInfo_ExpiryDate.append(locale_gen.date_between_dates(
            date_start = datetime.date(2022, 1, 1),
            date_end = datetime.date(2027, 1, 1)))
        PaymentInfo_AccountNbr.append(locale_gen.credit_card_number())
        WorkEmail.append(locale_gen.company_email())
        
    DOCS_ExpiryDate.append(locale_gen.date_between_dates(
        date_start = datetime.date(2022, 1, 1),
        date_end = datetime.date(2027, 1, 1)))
    

Combined.insert(13, 'FirstName', FirstName)
Combined.insert(14, 'DOB', DOB)
Combined.insert(18, 'FreeEmail', FreeEmail)
Combined.insert(19, 'WorkEmail', WorkEmail)
Combined.insert(20, 'DOCS_ExpiryDate', DOCS_ExpiryDate)
Combined.insert(21, 'PaymentInfo_VendorCode', PaymentInfo_VendorCode)
Combined.insert(22, 'PaymentInfo_ExpiryDate', PaymentInfo_ExpiryDate)
Combined.insert(23, 'PaymentInfo_AccountNbr', PaymentInfo_AccountNbr)

In [494]:
Combined

,HHID,P_ID,P_AGE,AgeRange,AgeGroup,P_GENDER,HH_num,HH_ISO,IATA_O,GenderHOH,...,Surname,Address,FreeEmail,WorkEmail,DOCS_ExpiryDate,PaymentInfo_VendorCode,PaymentInfo_ExpiryDate,PaymentInfo_AccountNbr,PostCode,Country
0,POI_0,POI_0_1,36,35-39,AGE8,M,0,IRL,DUB,M,...,Ó Lainn,"11971 Mac Thámhais Passage\nIngridton, AK 53359",graceo-fuada@hotmail.com,o-cuirreainangeline@o.com,2024-09-18,VISA 16 digit,2022-03-29,4461680422278849,17045,Republic of Ireland
1,POI_0,POI_0_2,15,15-19,AGE4,M,0,IRL,DUB,M,...,Ó Lainn,"11971 Mac Thámhais Passage\nIngridton, AK 53359",o-madagainrita@gmail.com,-,2023-12-12,-,-,-,17045,Republic of Ireland
2,POI_1,POI_1_1,13,10-14,AGE3,M,1,DEU,MUC,M,...,Zirme,Irmtraud-Dietz-Allee 0\n50901 Wetzlar,qmac-aonghais@yahoo.com,-,2022-03-16,-,-,-,30274,Germany
3,POI_2,POI_2_1,29,25-29,AGE6,F,2,ROU,OTP,F,...,Tudor,"Intrarea Ghenadie Manole\nAnina, 245855",o-goillidheronan@gmail.com,leigh04@o.com,2024-04-26,Diners Club / Carte Blanche,2025-04-16,379876002339931,904976,Romania
4,POI_2,POI_2_2,12,10-14,AGE3,M,2,ROU,OTP,F,...,Tudor,"Intrarea Ghenadie Manole\nAnina, 245855",o-conghamhnasuzanne@hotmail.com,-,2024-05-10,-,-,-,904976,Romania
5,POI_3,POI_3_1,46,45-49,AGE10,M,3,SWE,ARN,M,...,Eriksson,Idrottsvägen 422\n28628 Sundsvall,o-flanagainthaddeus@gmail.com,fo-crotaigh@o.com,2022-09-13,VISA 16 digit,2025-01-01,4877664956346,54847,Sweden
6,POI_3,POI_3_2,7,5-9,AGE2,M,3,SWE,ARN,M,...,Eriksson,Idrottsvägen 422\n28628 Sundsvall,-,-,2022-10-02,-,-,-,54847,Sweden
7,POI_4,POI_4_1,9,5-9,AGE2,F,4,LVA,RIX,F,...,Arroyo,"0769 Brown Mall\nEricmouth, SC 51648",-,-,2022-01-10,-,-,-,24248,United States
8,POI_5,POI_5_1,51,50-54,AGE11,M,5,CHE,ZRH,M,...,Pasquier,rue Nusslé\n4674 Chatriand-sur-Pachoud,mac-robhartaighelena@hotmail.com,flannandoghair@mac.com,2023-02-19,VISA 19 digit,2025-01-28,3532683328094796,5788,Switzerland
9,POI_5,POI_5_2,10,10-14,AGE3,M,5,CHE,ZRH,M,...,Pasquier,rue Nusslé\n4674 Chatriand-sur-Pachoud,o-laoidermott@hotmail.com,-,2024-03-13,-,-,-,5788,Switzerland


In [499]:
book_POI_2

,initID,list_of_passengers,stayDay,route,num_in_party,fixRoute,Leg_fixRoute,LegHH_ISO,Airport_Itinerary_Out,full_routes,outboundDate,returnDate,outboundFlights,Airport_Itinerary_Ret,return_routes,returnFlights
1,MUC-LGW-POI-ID0,[POI_1_1],NaN,MUC-LGW,1,MUC->LGW,1.0,DEU-GBR,"[MUC, LGW]",[MUC-LGW],2019-03-20 00:00:00+00:00,NaT,[141248],"[LGW, MUC]",[LGW-MUC],[nan]
2,OTP-LGW-POI-ID0,"[POI_2_1, POI_2_2]",NaN,OTP-LGW,2,OTP->LGW,1.0,ROU-GBR,"[OTP, LGW]",[OTP-LGW],2019-03-10 00:00:00+00:00,NaT,[74802],"[LGW, OTP]",[LGW-OTP],[nan]
3,ARN-BRS-POI-ID0,"[POI_3_1, POI_3_2]",NaN,ARN-BRS,2,ARN->AMS->BRS,2.0,SWE-GBR,"[ARN, AMS, BRS]","[ARN-AMS, AMS-BRS]",2019-03-21 00:00:00+00:00,NaT,"[151717, 153090]","[BRS, AMS, ARN]","[BRS-AMS, AMS-ARN]","[nan, nan]"
4,RIX-NOC-POI-ID0,[POI_4_1],NaN,RIX-NOC,1,RIX->STN->NOC,2.0,LVA-IRL,"[RIX, STN, NOC]","[RIX-STN, STN-NOC]",2019-03-07 00:00:00+00:00,NaT,"[53398, nan]","[NOC, STN, RIX]","[NOC-STN, STN-RIX]","[nan, nan]"
8,ZAG-HAM-POI-ID0,"[POI_8_1, POI_8_2]",NaN,ZAG-HAM,2,ZAG->LHR->HAM,2.0,HRV-DEU,"[ZAG, LHR, HAM]","[ZAG-LHR, LHR-HAM]",2019-03-26 00:00:00+00:00,NaT,"[184710, 185978]","[HAM, LHR, ZAG]","[HAM-LHR, LHR-ZAG]","[nan, nan]"
9,OTP-LHR-POI-ID0,[POI_9_1],NaN,OTP-LHR,1,OTP->LHR,1.0,ROU-GBR,"[OTP, LHR]",[OTP-LHR],2019-03-31 00:00:00+00:00,NaT,[217108],"[LHR, OTP]",[LHR-OTP],[nan]


In [505]:
flight_POI_2.to_csv('buildXML/flights_POI.csv')
book_POI_2.to_csv('buildXML/bookings_POI.csv')
df_HH.to_csv('buildXML/HH_POI.csv')
Combined.to_csv('buildXML/P_POI.csv')

In [510]:
flight_POI_2[flight_POI_2['flight_id'] == 184710]['bookings'].tolist()

[['ZAG-LHR-group-ID1',
  'ZAG-LHR-group-ID2',
  'ZAG-LHR-group-ID13',
  'ZAG-LHR-business-ID3',
  'ZAG-HAM-business-ID1',
  'ZAG-ARN-leisure-ID13',
  'ZAG-HAM-leisure-ID11',
  'ZAG-ORD-leisure-ID6',
  'ZAG-LHR-leisure-ID98',
  'ZAG-LHR-leisure-ID113',
  'ZAG-LHR-leisure-ID125',
  'ZAG-LHR-leisure-ID167',
  'ZAG-LHR-leisure-ID168',
  'ZAG-HAM-leisure-ID38',
  'ZAG-LHR-leisure-ID268',
  'ZAG-LHR-leisure-ID283',
  'ZAG-LHR-leisure-ID297',
  'ZAG-GOT-leisure-ID15',
  'ZAG-LHR-leisure-ID333',
  'ZAG-ARN-leisure-ID101',
  'ZAG-LHR-leisure-ID365',
  'ZAG-ARN-leisure-ID112',
  'ZAG-LHR-leisure-ID385',
  'ZAG-LHR-leisure-ID388',
  'ZAG-ORD-leisure-ID30',
  'ZAG-GOT-leisure-ID28',
  'ZAG-ORD-leisure-ID36',
  'ZAG-LHR-leisure-ID454',
  'MAD-ZAG-business-ID0',
  'MAD-ZAG-business-ID4',
  'LHR-ZAG-business-ID3',
  'LHR-ZAG-business-ID9',
  'DUB-ZAG-leisure-ID4',
  'MAD-ZAG-leisure-ID2',
  'MAD-ZAG-leisure-ID17',
  'MAD-ZAG-leisure-ID26',
  'MAD-ZAG-leisure-ID41',
  'LHR-ZAG-leisure-ID10',
  'LHR-ZA